In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Notation
# N = number of samples
# T = sequence length
# D = number of input features
# M = number of hidden units
# K = number of output units

In [3]:
# Make some data
N = 1
T = 10
D = 3
M = 5
K = 2
X = np.random.randn(N, T, D)

In [4]:
# Make an RNN
class SimpleRNN(nn.Module):
  def __init__(self, n_inputs, n_hidden, n_outputs):
    super(SimpleRNN, self).__init__()
    self.D = n_inputs
    self.M = n_hidden
    self.K = n_outputs
    self.rnn = nn.RNN(
        input_size=self.D,
        hidden_size=self.M,
        nonlinearity='tanh',
        batch_first=True)
    self.fc = nn.Linear(self.M, self.K)
  
  def forward(self, X):
    # initial hidden states
    h0 = torch.zeros(1, X.size(0), self.M)

    # get RNN unit output
    out, _ = self.rnn(X, h0)

    # we only want h(T) at the final time step
    #out = self.fc(out[:, -1, :])
    out = self.fc(out)
    return out

In [5]:
# Instantiate the model
model = SimpleRNN(n_inputs=D, n_hidden=M, n_outputs=K)

In [6]:
# Get the output
inputs = torch.from_numpy(X.astype(np.float32))
out = model(inputs)
out

tensor([[[ 0.1558,  0.5023],
         [-0.2727,  0.0524],
         [ 0.4168,  0.1936],
         [ 0.0547,  0.2806],
         [ 0.1306,  0.1462],
         [ 0.0186, -0.1402],
         [ 0.4313,  0.2753],
         [ 0.1836,  0.5135],
         [-0.0223, -0.2090],
         [-0.0542, -0.0880]]], grad_fn=<AddBackward0>)

In [7]:
out.shape

torch.Size([1, 10, 2])

In [8]:
# Save for later
Yhats_torch = out.detach().numpy()

In [9]:
W_xh, W_hh, b_xh, b_hh = model.rnn.parameters()

In [10]:
W_xh.shape

torch.Size([5, 3])

In [11]:
W_xh

Parameter containing:
tensor([[-0.1874, -0.2639,  0.1810],
        [ 0.3067,  0.4029,  0.2249],
        [-0.0632,  0.1884,  0.3253],
        [-0.3286, -0.1887,  0.2735],
        [-0.3911,  0.1312,  0.4388]], requires_grad=True)

In [12]:
W_xh = W_xh.data.numpy()
W_xh

array([[-0.1874365 , -0.26388124,  0.1809541 ],
       [ 0.30674154,  0.40291274,  0.22489743],
       [-0.06315278,  0.18840556,  0.32530475],
       [-0.3285669 , -0.18871088,  0.2734532 ],
       [-0.391126  ,  0.13123669,  0.43876562]], dtype=float32)

In [13]:
b_xh = b_xh.data.numpy()
W_hh = W_hh.data.numpy()
b_hh = b_hh.data.numpy()

In [14]:
# Did we do it right?
W_xh.shape, b_xh.shape, W_hh.shape, b_hh.shape

((5, 3), (5,), (5, 5), (5,))

In [15]:
# Now get the FC layer weights
Wo, bo = model.fc.parameters()

In [16]:
Wo = Wo.data.numpy()
bo = bo.data.numpy()
Wo.shape, bo.shape

((2, 5), (2,))

In [ ]:
# See if we can replicate the output
h_last = np.zeros(M) # initial hidden state
x = X[0] # the one and only sample
Yhats = np.zeros((T, K)) # where we store the outputs

for t in range(T):
  h = np.tanh(x[t].dot(W_xh.T) + b_xh + h_last.dot(W_hh.T) + b_hh)
  y = h.dot(Wo.T) + bo # we only care about this value on the last iteration
  Yhats[t] = y
  
  # important: assign h to h_last
  h_last = h

# print the final output
print(Yhats)

[[ 0.16283656  0.41041298]
 [ 0.27156185  0.7004437 ]
 [ 0.13568808  0.80314295]
 [-0.25995046  0.90357084]
 [ 0.0573792   0.73060827]
 [ 0.54373072  0.70356239]
 [ 0.69360861  0.6089178 ]
 [ 0.40267544  0.72920445]
 [ 0.45428415  0.79174287]
 [ 0.05785302  1.08336535]]


In [ ]:
# Check
np.allclose(Yhats, Yhats_torch)

True